In [1]:
with open('Day07input.txt') as f:
    lines = f.readlines()

#something to remove \n 
str_lines = []
for l in lines:
    str_lines.append(l[0:-1])#''.join(lines[0:len(lines[0])-2])



# --- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:


In [2]:
test_data = [
    "$ cd /",
    "$ ls",
    "dir a",
    "14848514 b.txt",
    "8504156 c.dat",
    "dir d",
    "$ cd a",
    "$ ls",
    "dir e",
    "29116 f",
    "2557 g",
    "62596 h.lst",
    "$ cd e",
    "$ ls",
    "584 i",
    "$ cd ..",
    "$ cd ..",
    "$ cd d",
    "$ ls",
    "4060174 j",
    "8033020 d.log",
    "5626152 d.ext",
    "7214296 k"]


The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.


In [3]:
from anytree import Node, RenderTree
import re
top_level = Node("/")
nodes = {}
nodes['/'] = (top_level, "dir")


In [4]:
def run_input(elf_data):
    nodes.clear()
    nodes['/'] = (top_level, "dir")
    current_node = nodes['/'][0]
    path = "/"
    for d in elf_data[1:]:
        if d[0] == "$":
            (current_node, path) = execute_instruction(d[2:], current_node, path)
        else:
            process_output(d, current_node, path)

def execute_instruction(d, current_node, path):
    if d == "ls":
        return (current_node, path)
    elif d[0:2] == "cd":
        if d[3:] == "..":
            #print(path)
            #x = path.rfind("/")
            #print(path[:x])
            return (current_node.parent, path[:path.rfind("/")])
        elif d[3:] == "/":
            return (top_level, "/")
        else: #cd directory
            return (nodes[(path+"/"+d[3:])][0], (path+"/"+d[3:]))
        
def process_output(d, current_node, path):
    command = d.split()   
    newNode = Node((path+"/"+command[1]), parent = current_node)
    nodes[(path+"/"+command[1])] = (newNode, command[0])
                


In [5]:
#run_input(test_data)


In [6]:
def print_tree():
    for pre, fill, node in RenderTree(top_level):
        print("%s%s" % (pre, node.name))


In [7]:

#print_tree()


Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
cd / switches the current directory to the outermost directory, /.
ls means list. It prints out all of the files and directories immediately contained by the current directory:
123 abc means that the current directory contains a file named abc with size 123.
dir xyz means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)


In [8]:
def calc_dir_size(node):
    if node[1] != "dir":
        return int(node[1])
    else: 
        sum = 0
        for c in node[0].children:
            sum = sum+calc_dir_size(nodes[c.name])
        return sum



In [9]:
#print(nodes["//d"])
#i = calc_dir_size(nodes["//d"])
#print(i)


The total sizes of the directories above can be found as follows:


In [10]:


#The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
#print(calc_dir_size(nodes["//a/e"]))  

#The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
#print(calc_dir_size(nodes["//a"]))  

#Directory d has total size 24933642.
#print(calc_dir_size(nodes["//d"]))  


#As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
#print(calc_dir_size(nodes["/"]))  



To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)



In [11]:
def calc_sum_sub(upper_limit):
    total = 0
    for n in nodes:
        if nodes[n][1] == "dir":
            x = calc_dir_size(nodes[n])
            if x<= upper_limit:
                total = total + x                
    return total



In [12]:
#print(calc_sum_sub(100000))

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

now do it with the real data

In [13]:
run_input(str_lines)


In [14]:
print_tree()


/
├── //bzgf
│   └── //bzgf/htczftcn.bdr
├── //dngdnvv.qdf
├── //fhhwv
│   ├── //fhhwv/bjml
│   │   ├── //fhhwv/bjml/bzmwfgf.wwp
│   │   ├── //fhhwv/bjml/cwghv
│   │   │   ├── //fhhwv/bjml/cwghv/htczftcn
│   │   │   ├── //fhhwv/bjml/cwghv/nflgvsgz
│   │   │   ├── //fhhwv/bjml/cwghv/svfqpfm.bgs
│   │   │   ├── //fhhwv/bjml/cwghv/tdgw
│   │   │   │   └── //fhhwv/bjml/cwghv/tdgw/htczftcn.cnl
│   │   │   ├── //fhhwv/bjml/cwghv/tzrz.hms
│   │   │   └── //fhhwv/bjml/cwghv/zmm.ndc
│   │   ├── //fhhwv/bjml/dngdnvv.qdf
│   │   ├── //fhhwv/bjml/jbc.zgp
│   │   ├── //fhhwv/bjml/mcqbcttc.mwr
│   │   ├── //fhhwv/bjml/pffdg
│   │   │   ├── //fhhwv/bjml/pffdg/bbqh.wzf
│   │   │   ├── //fhhwv/bjml/pffdg/bgzdv
│   │   │   │   └── //fhhwv/bjml/pffdg/bgzdv/srzlfd.hnq
│   │   │   ├── //fhhwv/bjml/pffdg/bhrw.jls
│   │   │   ├── //fhhwv/bjml/pffdg/hqwmfdj
│   │   │   │   ├── //fhhwv/bjml/pffdg/hqwmfdj/dzpgcrn.rbj
│   │   │   │   ├── //fhhwv/bjml/pffdg/hqwmfdj/mrrngcdr
│   │   │   │   └── //fhhwv/bjml/pffdg/

In [16]:
xx = calc_sum_sub(100000)
print(xx)


2104783


#--- Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.


In [17]:
total_disk_space = 70000000
space_needed = 30000000
max_space_allowed = total_disk_space - space_needed

In [19]:
current_space = calc_dir_size(nodes["/"])
print(current_space)


44965705


In [27]:
def calc_sum_sub2(lower_limit):
    current_best = calc_dir_size(nodes["/"])
    for n in nodes:
        if nodes[n][1] == "dir":
            x = calc_dir_size(nodes[n])
            if x>= lower_limit:
                if x<current_best:
                    current_best = x
    return current_best

In [28]:
if current_space > max_space_allowed:
    to_free_up = current_space- max_space_allowed
    print(calc_sum_sub2(to_free_up))

5883165



In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

Delete directory e, which would increase unused space by 584.
Delete directory a, which would increase unused space by 94853.
Delete directory d, which would increase unused space by 24933642.
Delete directory /, which would increase unused space by 48381165.
Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?